In [194]:
import pandas as pd
import numpy as np
def get_hdf_keys(dir):
    with pd.HDFStore(dir) as store:
        return [e[1:] for e in store.keys()]

### Sort, Type casting

In [195]:
rename_dict = {'이름':'name', '이미지링크':'image', '최저가로고':'shop_logo', '최저가브랜드명':'shop_name', '최저가링크':'shop_link'}
dirs = [
    'danawa_crawling_data_cmoh.h5',
    'danawa_crawling_data_seokjin.h5',
    'danawa_crawling_data_jinu.h5',
]

dfs_list = [{key : pd.read_hdf(dir, key) for key in get_hdf_keys(dir)} for dir in dirs]
for dfs in dfs_list:
    for key, df in dfs.items():
        df.rename(columns=rename_dict, inplace=True)

# strange data
del dfs_list[1]['SSD_detail']

# for dfs in dfs_list:
#     for key, df in dfs.items():
#         if key.endswith('detail'):
#             print(key, df.columns)

In [196]:
key_priority_aux1 = ['CPU', 'RAM', 'VGA', 'MBoard', 'SSD', 'HDD', 'Power', 'Cooler', 'Case', 'Monitor', 'Speaker', 'Headphone', 'Earphone', 'Headset', 'Keyboard', 'Mouse', 'Laptop']
key_priority_aux2 = ['', '_detail', '_review']
key_priority = [a + b for a in key_priority_aux1 for b in key_priority_aux2]
result_key = [key for dfs in dfs_list for key in dfs.keys()]
result_key = sorted(list(set(result_key)), key=lambda x : key_priority.index(x))
# result_key

In [197]:
result = {}
for key in result_key:
    for dfs in dfs_list:
        if key in dfs.keys():
            if key not in result.keys():
                result[key] = dfs[key]
            else:
                result[key] = result[key].join(dfs[key], how='inner', lsuffix='', rsuffix='_tmp')
                result[key].drop(columns=[label for label in result[key].columns if label.endswith('_tmp')], inplace=True)
    

In [198]:
for key in result_key:
    if key.endswith('review'):
        result[key].set_index('id', inplace=True)

In [199]:
for key in result_key:
    if key.endswith('detail') or key.endswith('review'):
        idx = list(set(result[key].index) - set(result[key[:-7]].index))
        result[key].drop(idx, inplace=True)

In [200]:
for key in result_key:
    if key.endswith('detail'):
        print(list(set(result['CPU_detail'].index) ^ set(result['CPU'].index)))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [201]:
for key, df in result.items():
    print(key)
    print(df)

CPU
         id_validator
id                   
12621851         True
12625265         True
15137684         True
13763867         True
12872324         True
...               ...
5449121          True
1049525          True
686573           True
5651804          True
5132495          True

[890 rows x 1 columns]
CPU_detail
                                           name  \
id                                                
12621851          AMD 라이젠5-4세대 5600X (버미어) (정품)   
12625265     AMD 라이젠5-4세대 5600X (버미어) (멀티팩(정품))   
15137684   AMD 라이젠5-4세대 5600X (버미어) (멀티팩(프리즘팩))   
13763867          AMD 라이젠5-4세대 5600X (버미어) (벌크)   
12872324        AMD 라이젠5-4세대 5600X (버미어) (해외구매)   
...                                         ...   
686573                   인텔 펜티엄 E2200 (콘로) (중고)   
587936                 인텔 코어2듀오 E4500 (콘로) (중고)   
15814061  인텔 코어i5-10세대 10400 ES (코멧레이크S) (해외구매)   
5651804        인텔 코어i3-7세대 7350K (카비레이크) (해외구매)   
5132495            인텔 제온 E3-1245V6 (카비레이크) (정품)   

           

In [202]:
result_dir = 'danawa_crawling_data_join.h5'
for key, df in result.items():
    df.to_hdf(result_dir, key)

C:\Users\seokj\AppData\Local\Temp\ipykernel_13492\2724501062.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->boolean,key->block0_values] [items->Index(['id_validator'], dtype='object')]

  df.to_hdf(result_dir, key)
C:\Users\seokj\AppData\Local\Temp\ipykernel_13492\2724501062.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['name', 'image', 'shop_link', 'shop_name', 'shop_logo', '제조회사', '등록년월',
       'AMD CPU종류', '소켓 구분', '세대 구분', '출시일', '제조 공정', '사양코어 수', '사양쓰레드 수',
       '사양기본 클럭', '사양최대 클럭', '사양L2 캐시', '사양L3 캐시', '사양TDP', '사양PCIe 버전',
       '메모리 사양메모리 규격', '메모리 사양메모리 클럭', '그래픽 사양내장그래픽', '기술 지원StoreMI',
       '기술 지원Ryzen Master', '기술 지원VR Ready 프리미엄', '구성패키지 형태', '구성쿨러',
       'KC인증적합성평가인증', 'KC인증안전확인인증', '인텔 CPU종류', '사양연산 체계', '사양버스 속도',
      